In [1]:
# make csv ready to datasets fromat
import pandas as pd
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
#import evaluate


d_a = pd.read_csv("data/twitter_data/labels_tweets_dropna.csv")

from sklearn.model_selection import train_test_split
d_a['label'] = d_a['Cyberbullying'] + d_a['Insult'] + d_a['Threat']
d_a['label'] = d_a['label'].apply(lambda x: 1 if x > 0 else 0)
d_a['billed'] = d_a['Tweet'].apply(lambda x: 1 if 'http' in x else 0)
d_a_ub = d_a[d_a['billed']==0]
nt = d_a[d_a.billed==0].label.sum()

d_a_balanced = d_a_ub[d_a_ub.label == 0].iloc[:nt,:].append(d_a_ub[d_a_ub.label ==1]).sample(frac=1)
d_a_r = d_a_balanced[['Tweet','label']]

d_train, d_test = train_test_split(d_a_r, test_size=0.2, random_state=42)
d_test1, d_val = train_test_split(d_test, test_size=0.2, random_state=42)

d_train.to_csv("data/twitter_data/tweet_train.csv",index=False)
d_test1.to_csv("data/twitter_data/tweet_test.csv",index=False)
d_val.to_csv("data/twitter_data/tweet_val.csv",index=False)

In [16]:
#from pandas import option_context
#with option_context('display.max_colwidth', 400):
 #   display(d_a_r.head(20))

In [3]:
data_files = {"train": "data/twitter_data/tweet_train.csv", "test": "data/twitter_data/tweet_test.csv", "val": "data/twitter_data/tweet_val.csv"}
dataset = load_dataset("csv", data_files=data_files)


Using custom data configuration default-5660e830f2ae2ba3


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/bruger/.cache/huggingface/datasets/csv/default-5660e830f2ae2ba3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
training_args = TrainingArguments(output_dir="test_trainer")

In [8]:
#metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return sum(predictions==labels)/len(labels)
    #return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=50)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\bruger\miniconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6
  Number of trainable parameters = 108311810


  0%|          | 0/6 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.7921687364578247, 'eval_accuracy': 0.3, 'eval_runtime': 18.1396, 'eval_samples_per_second': 0.551, 'eval_steps_per_second': 0.11, 'epoch': 1.0}


...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


  0%|          | 0/2 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.7768245935440063, 'eval_accuracy': 0.4, 'eval_runtime': 18.8829, 'eval_samples_per_second': 0.53, 'eval_steps_per_second': 0.106, 'epoch': 3.0}
{'train_runtime': 280.1515, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.021, 'train_loss': 0.6408406098683676, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=0.6408406098683676, metrics={'train_runtime': 280.1515, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.021, 'train_loss': 0.6408406098683676, 'epoch': 3.0})

In [ ]:
trainer.save_model('bert_kat_trained')

In [13]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
dataset['val']

Dataset({
    features: ['Tweet', 'label'],
    num_rows: 279
})

In [15]:
model.eval(dataset['val'])

TypeError: eval() takes 1 positional argument but 2 were given